# 201810794 이은후

In [1]:
%%writefile src/Rsp.sol
pragma solidity 0.5.1;
contract Rsp {
    // 1: rock, 2: scissors, 3: paper
    uint computer;
    uint user;
    string result;
    uint resultNum;
    uint betMoney;
    uint amount = 1000 wei;
    address owner;
    address accountB;
    mapping (address => uint) balanceOf;
    constructor() public {
        owner = msg.sender;
    }
    function deposit(uint input) payable public onlyOwner {
        require(msg.value == input);
        balanceOf[msg.sender] += input;
    }
    function setA() public {
        uint nonce=0;
        computer = uint(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        computer += 1;
        nonce++;
        require(balanceOf[msg.sender] >= amount);
        balanceOf[msg.sender] -= amount;
        betMoney += amount;
    }
    function whatANum() public view returns(uint) {
        return computer;
    }
    function setBInfo(address addrB) public {
        accountB = addrB;
        balanceOf[accountB] = accountB.balance;
    }
    function setB(uint _user) public {
        user = _user;
        //balanceOf[accountB] = accountB.balance;
        balanceOf[accountB] -= amount;
        betMoney += amount;
    }
    function getBetMoney() public view returns(uint) {
        return betMoney;
    }
    function play() public {
        if (computer == user) {
            result = "tie";
        }
        else if ((computer == 1 && user == 2) || (computer == 2 && user == 3) || (computer == 3 && user == 1)) {
            result = "A wins";
            resultNum = 1;
        }
        else if ((computer == 1 && user == 3) || (computer == 2 && user == 1) || (computer == 3 && user == 2)) {
            result = "B wins";
            resultNum = 2;
        }
    }
    function getMatchResult() public view returns(string memory) {
        return result;
    }
    function distributeBetAmount() public payable {
        if (resultNum == 1) // A(computer) wins
        {
            balanceOf[msg.sender] += betMoney;
            betMoney -= betMoney;
        }
        else if (resultNum == 2) // B(user) wins
        {
            balanceOf[accountB] += betMoney;
            betMoney -= betMoney;
        }
    }
    function getBalance() public view returns (uint, uint, uint) {
        return (balanceOf[msg.sender], balanceOf[accountB], address(this).balance);
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/Rsp.sol


In [3]:
!solc --abi --bin --gas src/Rsp.sol


======= src/Rsp.sol:Rsp =======
Gas estimation:
construction:
   40866 + 560400 = 601266
external:
   deposit(uint256):	21011
   distributeBetAmount():	26924
   getBalance():	1506
   getBetMoney():	528
   getMatchResult():	infinite
   play():	infinite
   setA():	82150
   setB(uint256):	61417
   setBInfo(address):	41510
   whatANum():	506
Binary: 
60806040526103e860055534801561001657600080fd5b5033600660006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610af2806100676000396000f3fe60806040526004361061009e576000357c01000000000000000000000000000000000000000000000000000000009004806309cdcf9b146100a357806312065fe0146100de5780631ab2aa94146101175780631fc376f7146101685780634a6863811461017f57806371b43fa91461020f57806387afb84f1461023a57806393e84cd914610265578063a0be28f51461027c578063b6b55f2514610286575b600080fd5b3480156100af57600080fd5b506100dc600480360360208110156100c657600080fd5b81019080803590602001909291905050506102b4565b00

### geth 사용하여 배포

In [4]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray  =[{"constant":false,"inputs":[{"name":"_user","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"addrB","type":"address"}],"name":"setBInfo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"whatANum","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getBetMoney","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin = "0x" + "60806040526103e860055534801561001657600080fd5b5033600660006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610af2806100676000396000f3fe60806040526004361061009e576000357c01000000000000000000000000000000000000000000000000000000009004806309cdcf9b146100a357806312065fe0146100de5780631ab2aa94146101175780631fc376f7146101685780634a6863811461017f57806371b43fa91461020f57806387afb84f1461023a57806393e84cd914610265578063a0be28f51461027c578063b6b55f2514610286575b600080fd5b3480156100af57600080fd5b506100dc600480360360208110156100c657600080fd5b81019080803590602001909291905050506102b4565b005b3480156100ea57600080fd5b506100f3610341565b60405180848152602001838152602001828152602001935050505060405180910390f35b34801561012357600080fd5b506101666004803603602081101561013a57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061040b565b005b34801561017457600080fd5b5061017d6104ee565b005b34801561018b57600080fd5b50610194610604565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156101d45780820151818401526020810190506101b9565b50505050905090810190601f1680156102015780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561021b57600080fd5b506102246106a6565b6040518082815260200191505060405180910390f35b34801561024657600080fd5b5061024f6106af565b6040518082815260200191505060405180910390f35b34801561027157600080fd5b5061027a6106b9565b005b610284610864565b005b6102b26004803603602081101561029c57600080fd5b8101908080359060200190929190505050610967565b005b8060018190555060055460086000600760009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254039250508190555060055460046000828254019250508190555050565b6000806000600860003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460086000600760009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020543073ffffffffffffffffffffffffffffffffffffffff1631925092509250909192565b80600760006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550600760009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163160086000600760009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000208190555050565b6000809050600342446040516020018083815260200182815260200192505050604051602081830303815290604052805190602001206001900481151561053157fe5b06600081905550600160008082825401925050819055508080600101915050600554600860003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054101515156105a057600080fd5b600554600860003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254039250508190555060055460046000828254019250508190555050565b606060028054600181600116156101000203166002900480601f01602080910402602001604051908101604052809291908181526020018280546001816001161561010002031660029004801561069c5780601f106106715761010080835404028352916020019161069c565b820191906000526020600020905b81548152906001019060200180831161067f57829003601f168201915b5050505050905090565b60008054905090565b6000600454905090565b6001546000541415610716576040805190810160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525060029080519060200190610710929190610a21565b50610862565b600160005414801561072a57506002600154145b806107455750600260005414801561074457506003600154145b5b8061075f5750600360005414801561075e575060018054145b5b156107bd576040805190810160405280600681526020017f412077696e730000000000000000000000000000000000000000000000000000815250600290805190602001906107af929190610a21565b506001600381905550610861565b60016000541480156107d157506003600154145b806107eb575060026000541480156107ea575060018054145b5b806108065750600360005414801561080557506002600154145b5b15610860576040805190810160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525060029080519060200190610856929190610a21565b5060026003819055505b5b5b565b600160035414156108d557600454600860003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550600454600460008282540392505081905550610965565b600260035414156109645760045460086000600760009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506004546004600082825403925050819055505b5b565b600660009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156109c357600080fd5b80341415156109d157600080fd5b80600860003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f10610a6257805160ff1916838001178555610a90565b82800160010185558215610a90579182015b82811115610a8f578251825591602001919060010190610a74565b5b509050610a9d9190610aa1565b5090565b610ac391905b80821115610abf576000816000905550600101610aa7565b5090565b9056fea165627a7a7230582018bfd90e3facb5b3855ea85fef4e3b36ae7c9c22496fa8ea28f85b47cbed62ea0029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/RspDeploy.js


마이닝하여 컨트랙주소 얻음

In [5]:
!node src/RspDeploy.js

Deploying the contract from 0x52CadD19F40f5F5f544F33FDE2f8FC118862E4F9
hash: 0x8f51a31c62ad58827ba2f3d029fb70f3454df54017248817e5fde131a9a88a6f
---> The contract deployed to: 0x9c3174d6894d519871b10C6F9012213425E43042


In [6]:
%%writefile src/RspUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_user","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"addrB","type":"address"}],"name":"setBInfo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"whatANum","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getBetMoney","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
async function doIt() {
    var rsp = new web3.eth.Contract(_abiArray, '0x9c3174d6894d519871b10C6F9012213425E43042');
    const accounts = await web3.eth.getAccounts();
    await rsp.methods.deposit(1000).send({from:accounts[0],gas:1000000,value:1000});
    await rsp.methods.setBInfo(accounts[1]).send({from:accounts[0],gas:1000000});
    console.log("<BEFORE game>");
    rsp.methods.getBalance().call().then(function(value){console.log(value);});
    await rsp.methods.setA().send({from:accounts[0],gas:1000000});
    console.log("\n");
    console.log("What A choose: ");
    rsp.methods.whatANum().call().then(function(value){console.log(value);});
    await rsp.methods.setB(3).send({from:accounts[0],gas:1000000});
    await rsp.methods.play().send({from:accounts[0],gas:1000000});
    rsp.methods.getMatchResult().call().then(function(value){console.log(value);});
    await rsp.methods.distributeBetAmount().send({from:accounts[0],gas:1000000});
    console.log("\n");
    console.log("<AFTER game>");
    rsp.methods.getBalance().call().then(function(value){console.log(value);});
}
doIt()

Overwriting src/RspUse.js


### 마이닝을 해주어 결과를 얻었음

# 1: rock, 2: scissors, 3: paper로 설정

### 1. B가 이긴 경우

In [7]:
!node src/RspUse.js

<BEFORE game>
Result { '0': '1000', '1': '269440000000000000999', '2': '1000' }


What A choose: 
1
B wins


<AFTER game>
Result { '0': '0', '1': '269440000000000001999', '2': '1000' }


### 2. 비긴 경우

In [8]:
!node src/RspUse.js

<BEFORE game>
Result { '0': '1000', '1': '269440000000000000999', '2': '2000' }


What A choose: 
3
tie


<AFTER game>
Result { '0': '0', '1': '269440000000000001999', '2': '2000' }


### 3. A가 이긴 경우

In [9]:
!node src/RspUse.js

<BEFORE game>
Result { '0': '1000', '1': '269440000000000000999', '2': '3000' }


What A choose: 
2
A wins


<AFTER game>
Result { '0': '2000', '1': '269439999999999999999', '2': '3000' }
